## Introduction or Motivation or Problem Statement
We are trying to train a facial recognition neural network. To be able to identify people in pictures 
with the people inside a database.


## Data Sources
The data used for training can be found at:

AT&T dataset - https://github.com/maticvl/dataHacker/blob/master/DATA/at%26t.zip 

(gray-scaled dataset with 400 samples)

lfw dataset - https://www.kaggle.com/datasets/atulanandjha/lfwpeople

(colored dataset with 13000+ images)


## Exploratory Data Analysis
Each class in the dataset is a person's name with 10 grey pictures of their portrait.

For data preparation the image transformed to size 100 x 100 to fit the model and changed into grey pictures and then turned into 1x100x100 tensor.

In SiameseNetworkDataset's getitem it choices at 50/50 chance to be in the same class or different class and by checking if's their folder's name is the same it produces a lebel of 1 or 0.


## Model
At the start we just wanted to make a basic facial recognition neural network to work, that's why we chose Siamese with contrastive loss with grey pictures, it is easy to understant and easy to implement.

The model used is the Siamese neural network, it has inputs as 2 images. Both images will pass through the same convolution neural network with fully connected layers at the end to produce 2 values as output. And the euclidean distance between the two images is the dissimilaity score (because both pictures went through the same neural network and if the images are in the same class it should have small distance score).

In [ ]:
#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

# Showing images
def imshow(img, text=None):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
        
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

# Plotting data
def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

class SiameseNetworkDataset(Dataset):
    def __init__(self,imageFolderDataset,transform=None):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)

        #We need to approximately 50% of images to be in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #Look untill the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] == img1_tuple[1]:
                    break
        else:

            while True:
                #Look untill a different class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] != img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])

        img0 = img0.convert("L")
        img1 = img1.convert("L")

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1, torch.from_numpy(np.array([int(img1_tuple[1] != img0_tuple[1])], dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

# Load the training dataset
folder_dataset = datasets.ImageFolder(root="./data/faces/training/")

# Resize the images and transform to tensors
transformation = transforms.Compose([transforms.Resize((100,100)),
                                     transforms.ToTensor()
                                    ])

# Initialize the network
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset,
                                        transform=transformation)

# create siameseNetwork
class SiameseNetwork(nn.Module):

    def __init__(self):
        super(SiameseNetwork, self).__init__()

        # Setting up the Sequential of CNN Layers
        self.cnn1 = nn.Sequential(
            
            nn.Conv2d(1, 64, 3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(64, 64, 3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            
            nn.Conv2d(64, 128, 3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            
            nn.Conv2d(128, 256, 3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(256, 256, 3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            
            nn.Conv2d(256, 384, 3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(384, 384, 3, stride=1, padding=1),
            nn.ReLU(inplace=True)
            
        )

        # Setting up the Fully Connected Layers
        self.fc1 = nn.Sequential(
            nn.Linear(13824, 6200),
            nn.ReLU(inplace=True),
            
            nn.Linear(6200, 3100),
            nn.ReLU(inplace=True),
            
            nn.Linear(3100, 1024),
            nn.ReLU(inplace=True),
            
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            
            nn.Linear(256,2)
        )
        
    def forward_once(self, x):
        # This function will be called for both images
        # Its output is used to determine the similiarity
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        #output = torch.flatten(output, 1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        # In this function we pass in both images and obtain both vectors
        # which are returned
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        return output1, output2
    
# Define the Contrastive Loss Function
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
      # Calculate the euclidean distance and calculate the contrastive loss
      euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)

      loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                    (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


      return loss_contrastive

# Load the training dataset
train_dataloader = DataLoader(siamese_dataset,
                        shuffle=True,
                        batch_size=64)


net = SiameseNetwork()
net.load_state_dict(torch.load('greymodel_more_save.pt'))
net.eval()
criterion = ContrastiveLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.0005 )

counter = []
loss_history = [] 
iteration_number= 0

# Iterate throught the epochs
for epoch in range(50):
    # Iterate over batches
    for i, (img0, img1, label) in enumerate(train_dataloader, 0):
        
        # Zero the gradients
        optimizer.zero_grad()

        # Pass in the two images into the network and obtain two outputs
        output1, output2 = net(img0, img1)

        # Pass the outputs of the networks and label into the loss function
        loss_contrastive = criterion(output1, output2, label)

        # Calculate the backpropagation
        loss_contrastive.backward()

        # Optimize
        optimizer.step()

        # Every 10 batches print out the loss
        if i % 10 == 0 :
            print(f"Epoch number {epoch}\n Current loss {loss_contrastive.item()}\n")
            iteration_number += 10

            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())

show_plot(counter, loss_history)

0 ---------------------------------------------------------------------------------------
1 ---------------------------------------------------------------------------------------
2 ---------------------------------------------------------------------------------------
3 ---------------------------------------------------------------------------------------
4 ---------------------------------------------------------------------------------------
5 ---------------------------------------------------------------------------------------
6 ---------------------------------------------------------------------------------------
Epoch number 0
 Current loss 0.4860076904296875

Epoch number 1
 Current loss 0.8250916600227356

Epoch number 2
 Current loss 0.6730691194534302

Epoch number 3
 Current loss 0.6003469228744507

Epoch number 4
 Current loss 0.5684418678283691



In [ ]:
torch.save(net.state_dict(), 'greymodel_more_2_save.pt')

In [ ]:
model = SiameseNetwork()
model.load_state_dict(torch.load('greymodel_save.pt'))
model.eval()